In [0]:
host = dbutils.secrets.get(scope = "postgresql", key = "host")
database = dbutils.secrets.get(scope = "postgresql", key = "database")
user = dbutils.secrets.get(scope = "postgresql", key = "user")
password = dbutils.secrets.get(scope = "postgresql", key = "password")

In [0]:
dividends_table = (spark.read
  .format("postgresql")
  .option("dbtable", "public.dividend_history")
  .option("host", host)
  .option("port", "5432") 
  .option("database", database)
  .option("user", user) 
  .option("password", password)
  .load()
)

In [0]:
dividends_table.display()

In [0]:
from delta.tables import DeltaTable

table_name = "plstocks.bronze_dividend_history"
if spark.catalog.tableExists(table_name):
    ExistingDididendTable = DeltaTable.forName(spark, table_name)
    ExistingDididendTable.alias("existing") \
        .merge(
            dividends_table.alias("updates"),
            "existing.ticker = updates.ticker AND existing.dividend_year = updates.dividend_year"
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    dividends_table.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable(table_name)